In [1]:
import warnings, os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from glob import glob


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from aggmap import AggMap, loadmap
import seaborn as sns


import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)
    ber =  (fp / (fp + tn) + fn / (tp + fn)) * 0.5

    print('acc: %.3f, roc-auc: %.3f, BER: %.3f' % (acc,auc,ber))

    return acc, auc


def get_best_params(X, y):
    clf = lgb.LGBMClassifier(verbose=-1, force_col_wise=True)
    parameters = {'n_estimators': [ 50, 100, 200],
                  'max_depth': [4,  6, 10]}

    grid = GridSearchCV(clf, parameters, scoring='roc_auc', n_jobs=9, iid=False, cv=5, verbose=0)
    gres = grid.fit(X, y)
    return grid.best_params_

In [6]:
mp = loadmap('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S-G.mp')
lst = glob('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/*.h5')

n_fold = 5

res = []
for p in lst:

    print('#'*50 + ' %s ' % p + '#'*50 )
    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')
    print(p, dfy.value_counts().to_dict())

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 123)
    outer_idx = list(outer.split(dfx, dfy))
    
    fold_all = []
    for i, idx in enumerate(outer_idx):
        fold_num = "fold_%s" % str(i).zfill(2) 
        train_idx, test_idx = idx

        # subset feature selection by the fold change
        dfxy_train = dfx.iloc[train_idx].join(dfy.iloc[train_idx])
        abds = dfxy_train.groupby(dfy.name).mean()
        fc = abs(abds.iloc[0] - abds.iloc[1])
        selected = fc[fc > 0.5].index

        trainX = dfx[selected].iloc[train_idx].values
        trainY = dfy.iloc[train_idx].values

        testX = dfx[selected].iloc[test_idx].values
        testY = dfy.iloc[test_idx].values
        
        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best parameters by gridsearch \n")
        best_params = get_best_params(trainX, trainY)
        clf = lgb.LGBMClassifier(**best_params, verbose=-1, n_jobs=-1, force_col_wise=True)
        clf.fit(trainX, trainY)

        ## evaluation
        y_true = testY
        y_score = clf.predict_proba(testX)[:,1]
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        res.append([auc, i, p, len(selected), best_params])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))


################################################## /home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5 ##################################################
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5 {0: 291, 1: 222}

Input train and test X shape is (410, 173), (103, 173)  

Getting the best parameters by gridsearch 

acc: 0.631, roc-auc: 0.686, BER: 0.382

Input train and test X shape is (410, 137), (103, 137)  

Getting the best parameters by gridsearch 

acc: 0.660, roc-auc: 0.703, BER: 0.357

Input train and test X shape is (410, 145), (103, 145)  

Getting the best parameters by gridsearch 

acc: 0.631, roc-auc: 0.706, BER: 0.375

Input train and test X shape is (411, 148), (102, 148)  

Getting the best parameters by gridsearch 

acc: 0.647, roc-auc: 0.698, BER: 0.367

Input train and test X shape is (411, 165), (102, 165)  

Getting the best parameters by gridsearch 

acc: 0.627, roc-auc: 0.674, BER: 0.410
The a

In [7]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task', 'selected_features','best_params'])
dfres.to_csv('./FS_LGB_5FCV_results_stage.csv')
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold,selected_features
task,,,
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_KIRC_stage.h5,0.786948,2.0,3130.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5,0.693414,2.0,153.6
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_COAD_stage.h5,0.688957,2.0,529.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_UCEC_stage.h5,0.681924,2.0,2879.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_LUAD_stage.h5,0.640448,2.0,3732.2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_SKCM_stage.h5,0.631652,2.0,1588.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_test_BRCA_stage.h5,0.621638,2.0,304.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_LIHC_stage.h5,0.620322,2.0,1544.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_validate_LUSC_stage.h5,0.607094,2.0,217.0


In [8]:
dfr = dfres.groupby('task').mean().sort_values('score',ascending=False)
order = ['COAD','KIRC','LIHC','LUAD','SKCM','STAD','THCA','UCEC','LUSC','BRCA']

dfr.index = dfr.index.map(lambda x:x.split('_')[-2])
dfr = dfr.loc[order]